# HPE Ezmeral Runtime Enterprise API series  - Lab 2
## Deploying cloud native and non-cloud native applications, programmatically on Kubernetes clusters managed by the HPE Ezmeral Runtime Enterprise.

### **Lab workflow:**

In this lab:

1. As tenant user, you will first establish a valid login session with HPE Ezmeral Runtime Enterprise using the REST API.

2. You will then fetch the Kubeconfig file for your tenant working context using the REST API.
>Note: A kubeconfig file is used to access the Kubernetes cluster for your tenant working context

3. Finally, using a K8s API client, such as **kubectl** and the traditional Kubernetes YAML approach, you will deploy a simple cloud native *stateless*, microservices applications, and a non-cloud native distributed *stateful* AI/analytics **KubeDirector** application on **Kubernetes** cluster managed by the HPE Ezmeral Runtime Enterprise.
>Note: kubectl has been installed in the JupyterHub server by the Lab administrator.

**Definitions:**

- *KubeDirector:* also known as Kubernetes Director, is a key component of the HPE Ezmeral Runtime Enterprise. KubeDirector is an open source project initiated by HPE that enables the running of non-cloud native stateful monolithic analytics workloads on Kubernetes. In the HPE Ezmeral Runtime Enterprise, these applications generally refer to as a distributed, single-node or multi-node application **virtual cluster**. Each application virtual cluster node runs as **a container** in the Kubernetes cluster managed by HPE Ezmeral Runtime Enterprise.

- *Cloud native application:* Also known as the 12 Factor app, a modern application that leverages microservices architecture with loosely coupled services.

- *Non-cloud native application:* a multi-tier application with tightly coupled and interdependent services. 

- *Stateless application:* A stateless application is an application which does not require persistence of data nor an application state.

- *Stateful application:* A stateful application typically requires persistence of certain mountpoints across application cluster nodes rescheduling, restarts, upgrades, rollbacks. A stateful application can also need persistence of network identity (i.e.: hostname). 

#### Initialize the environment:

Let's first define the environment variables according to your HPE Ezmeral Runtime Enterprise user account and tenant name, and the HPE Ezmeral Runtime Enterprise API system endpoint.

**Make sure the password is set to the password you received in the registraton e-mail**

In [ ]:
#
# environment variables to be adjusted/verified by the student
#
student="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student{{ STDID }}" # your HPE ECP tenant login credentials - username 
password="{{ PASSSTU }}" # your HPE ECP tenant login credentials - password - Please check with password provided in your registration e-mail.
#
# fixed environment variables setup by the lab administrator - Please DO NOT MODIFY!!
#
gateway_host="{{ HPEECPGWNAME }}"
controller_endpoint="${gateway_host}:8080"
Internet_access="{{ JPHOSTEXT }}"

tenantname="K8sHackTenant" #case sensitive

helloWorldApp="hello-world-app.yaml" # the application manifest you will deploy in this lab
tensorFlowApp="tensorflow-notebook-config-cluster.yaml" # the kubedirector application cluster configuration TensorFlow

echo "your operation context is:" $username "on tenant" $tenantname 

#### Authenticate as a tenant user in the specified tenant:

> Note:  When you see a [*] next to the next action it means your execution step is busy working within the notebook.

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

#### Get the Kubeconfig file for your tenant working context:
The next step in deploying a containerized application in Kubernetes clusters managed by the HPE Ezmeral Runtime Enterprise is to get the kubeconfig file for your tenant working context. 

The HPE Ezmeral Runtime Enterprise REST API call below allows you to get the **kubeconfig file** used to access the Kubernetes cluster for your tenant user account based on your assigned role (tenant member) with the same result as if you had downloaded it from the HPE Ezmeral Runtime Enterprise UI.

In [ ]:
mkdir -p ~/.kube
ls ~/.kube

In [ ]:
rm ~/.kube/config
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ~/.kube/config

In [ ]:
cat ~/.kube/config

> Notice the kubeconfig file includes the valid token (session location) for the current session you previously established.

#### Check your working tenant context:

In [ ]:
kubectl config current-context

You can now send Kubernetes API requests using a K8s API client such as **kubectl** to deploy enterprise workloads to the kubernetes cluster using the privileges assigned to your tenant role. 


**Let's see this in action!**

## Deploying a simple cloud native stateless application

#### -1- Deploy Hello World containerized application:
You will deploy a simple Hello World application on the Kubernetes cluster made available to your tenant. The K8s API call requires the kubectl operation type (create or apply) and the application manifest (YAML file that describes the attributes of the application).  

The hello-world application is a **stateless** application because it does not require persistence of data and application state. The hello-world application is a very simple application that will return `Hello Kubernetes!` when accessed. 

As you are all sharing the same tenant context and Kubernetes cluster resources, let's make sure your application deployment name will be unique among the tenant users. Here we replace the string "example" with your "username".

In [ ]:
sed -i "s/example/${username}/g" $helloWorldApp
cat $helloWorldApp

The YAML file describes the application resources to deploy, such as the Kubernetes Deployment, the Pod, the container Docker image and port, and the NodePort service used to expose the application outside the Kubernetes cluster.

>Note: the port name prefix **http-** in the service object definition. With this prefix, the application service will get SSL termination at the HPE Ezmeral Runtime Enterprise Gateway. 

In [ ]:
kubectl apply -f $helloWorldApp

After a few seconds, you should get the response message to your K8s API request: *Deployment and service created* on the Kubernetes cluster.   

In the next steps, you will use kubectl commands in the context of your tenant user account and get the service endpoints of your application to connect to it.

#### -2- Get the Kubernetes Pod and Service for your deployed application:

In [ ]:
kubectl get pod -l run=hello-${username}

In [ ]:
kubectl describe service hello-world-service-${username}

#### -3- Get the gateway mapped service endpoint:
HPE Ezmeral Runtime Enterprise automatically maps the **NodePort Service** endpoint to the HPE Ezmeral Runtime Enterprise gateway (proxy) host.
Access to application services running in containers is proxied via the gateway host on a publicly-accessible IP address and a port number greater than 10000.

In [ ]:
myservice="hello-world-service-"${username}
appURL=$(kubectl describe service/"${myservice}" | grep gateway | awk '{print $3}')
appPort=$(echo $appURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
myapp_endpoint="https://$gateway_host:$appPort"
echo "Your application service endpoint re-mapped port is: "$appPort
echo "Your Intranet application service endpoint is: "$myapp_endpoint

#### -4- Check to see if your application is responding:
You should get the output "Hello Kubernetes!"

In [ ]:
curl -k -s "${myapp_endpoint}"

#### -5- Delete your stateless application deployment:
Delete your application deployment and services using the K8s API request below. The K8s API call requires the kubectl operation type (delete) and the application YAML manifest.
After a minute or so, you should get the message: deployment deleted and service deleted.
>Note: you may get message: _Error from server (InternalError): an error on the server has prevented the request from succeeding._ Please ignore it.

In [ ]:
kubectl delete -f $helloWorldApp

## Deploying a KubeDirector application

HPE has been working within the open source Kubernetes community to add capabilities that enable the running of **stateful** analytics workloads (e.g: data-intensive, AI/ML and analytics-processing distributed applications) on Kubernetes.  The open source project is known as **Kubernetes Director** or **KubeDirector** for short.   
With KubeDirector, enterprises can deploy all their enterprise applications on a common Kubernetes framework.
KubeDirector is implemented as a Kubernetes operator (a custom controller with a set of Custom Resources such as *KubeDirectorApp* and *KubeDirectorCluster*) by default on every Kubernetes cluster managed by the HPE Ezmeral Runtime Enterprise.

For more information about KubeDirector, check out the HPE Developer Community portal [here](https://developer.hpe.com/platform/hpe-ezmeral/home).

#### -1- List the registered KubeDirector Applications:
You can get the list of KubeDirector Applications (kdapps) registered with the Kubernetes cluster for your Tenant using the following kubectl command:

In [ ]:
kubectl get kdapp

A KubeDirector Application describes an application's **metadata**: the service roles that compose the application virtual cluster, the service endpoints port and port name prefix (that comes from the URL Scheme), the Docker images, the app setup packages, the cardinality (number of members in a role), and if appropriate, the root file system mountpoints (e.g.: /etc, /bin, /opt, /var, /usr) of the containers to persist beyond the life span of the containers. This means stateful applications that require one to write data to their root file systems can now successfully run on Kubernetes. 

Let's inspect a couple of KubeDirector Application type definitions: 

In [ ]:
kubectl describe kdapp spark245

In [ ]:
kubectl describe kdapp tensorflow-cpu-jupyter 

#### -2- Deploy an instance of the TensorFlow with Jupyter Notebook stateful application:
Let's deploy an instance of the AI/ML TensorFlow application, which is the easiest example to understand.

A configuration manifest YAML file is used to deploy **an instance** of that KubeDirector App by creating a KubeDirector virtual cluster (KubeDirectorCluster or kdcluster) Customer Resource. The configuration file is used to describe the attributes (size and resource requirements) of the virtual cluster application.

We need to make sure that instance name of the KubeDirector Application is unique among the tenant users. 

In [ ]:
sed -i "s/example/${username}/g" $tensorFlowApp
cat $tensorFlowApp

The configuration manifest specifies the application instance name, the KubeDirector application type, the application service roles to deploy, the number of members (nodes) in a role and compute size, as well as the storage size if the application requires persistent storage. 

>Note: The TensorFlow KubeDirector application variant used in this tutorial specifies a single node implementation of TensorFlow and it does not specify any persistent storage. Jupyter Notebook is embedded in this local implementation and it is used as interactive client to execute AI/ML programs.

Let's deploy the TensorFlow KubeDirector application now using the K8s API call below. The API call requires the **kubectl** operation type (create or apply) and the application YAML manifest.

In [ ]:
kubectl apply -f $tensorFlowApp

After a few seconds, you should get the response message to your K8s API request: *kubedirectorcluster/Your-instance-name created*.  

Once you get this message, go to the next step.

#### -3- Inspect the deployed KubeDirector application instance: 
Your application will be represented in the Kubernetes cluster by a custom resource of type **KubeDirectorCluster (kdcluster)**, with the name that was indicated inside the YAML file used to create it. **A KubeDirectorCluster resource is an instantiation of a KubeDirector application**.

In [ ]:
clusterName="tensorflow-cpu-jupyter-${username}"
kubectl get kdcluster $clusterName

After creating the instance of the KubeDirector Application, you can use the `kubectl describe kdcluster` command below to observe its status and the standard Kubernetes resources that compose the application virtual cluster (statefulsets, pods, services, persistent volume claim if any), as well as any events logged against it.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. 

> Note: The first time a virtual cluster of a given KubeDirector Application type is created, it may take some minutes to reach its "configured" state, as the relevant Docker image must be downloaded and imported.

In [ ]:
kubectl describe kdcluster $clusterName

In [ ]:
kubectl get all -l kubedirector.hpe.com/kdcluster=$clusterName

Your instance of the TensorFlow KubeDirector Application virtual cluster is made up of a **StatefulSet**, a **POD** (a cluster node) per service role member, a **NodePort Service** per service role member and a **headless service** for the application cluster.   

* The ClusterIP service is the headless service required by a Kubernetes StatefulSet to work. It maintains a stable POD network identity (i.e.: persistence of the hostname of the PODs across PODs rescheduling).
* The NodePort service exposes an application service outside the Kubernetes cluster. 

HPE Ezmeral Runtime Enterprise automatically maps the NodePort Service endpoint to the HPE Ezmeral Runtime Enterprise gateway (proxy) host.

#### -4- Get the gateway mapped application service endpoint:
To get a report on all services related to a specific virtual cluster, you can use a form of **kubectl describe** that matches against a value of the **kubedirector.hpe.com/kdcluster=YourClusterApplicationName** label.

In [ ]:
tfappURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway | awk '{print $3}')
tfappPort=$(echo $tfappURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
mytfapp_endpoint="https://$gateway_host:$tfappPort"
echo "Your application service endpoint re-mapped port is: "$tfappPort
echo "Your Intranet application service endpoint is: "$mytfapp_endpoint

Access to application services running in containers is proxied via the gateway host and a port number greater than 10000.

#### -5- Verify the application service is responding:

In [ ]:
curl -k -L -s -i "${mytfapp_endpoint}" | grep "HTTP/1.1"

#### -6- Fetch the authentication token of TensorFlow Jupyter Notebook: 

Execute this command to get the authentication token for your Jupyter Notebook cluster: `kubectl exec pod-name jupyter notebook list` and use the token to login to your TensorFlow Jupyter notebook:

In [ ]:
mytfapp_pod=$(kubectl get pod -l kubedirector.hpe.com/kdcluster=${clusterName} | grep tensorflow | awk '{print $1}') 
mytfapp_token=$(kubectl exec ${mytfapp_pod} -- jupyter notebook list)
mytfToken=$(echo $mytfapp_token | cut -d'?' -f 2 | cut -d':' -f 1) # extract the Token value
echo "Your TensorFlow Jupyter Notebook login token is:" $mytfToken

#### -7- Access your application via the Network Service port from your browser:

The Jupyter Notebook cluster is typically used by Data Scientists and ML engineers to develop model and train, test, validate their models. 
Users accesses the Jupyter Notebook web UI via the Network Service port. In this lab, you access the application network service through our Internet NAT firewall on the gateway re-mapped port.

Execute the cell code below to obtain your network service URL.

In [ ]:
echo "Your application network service URL is:" "https://$Internet_access:$tfappPort?$mytfToken"

Click on the network service URL above. This should open a new tab in your browser and connect you to your Jupyter Notebook application.

>Note: You can also do a copy (CTRL-C) of the URL after execution of the cell code above and paste it (CTRL-V) on a new tab in your browser.

![TensorFlow Jupyter Notebook](Pictures/kubedirector-tf-jupyterNotebook.png)

#### -8- Go through some cleanup:

* First, delete the virtual cluster instance of the KubeDirector Application using the K8s API call below. The K8s API call requires the **kubectl** operation type (delete) and the application (YAML) manifest.

> Note: Deleting the KubeDirectorCluster resource will automatically delete all resources (for example the pods, services, statefulset) that compose the application virtual cluster.

In [ ]:
kubectl delete -f $tensorFlowApp

* Although sessions have a time to live (TTL) of 24 hours, it is a best practice in REST API programming to cleanup and delete those sessions when done. We can use a DELETE /api/v2/session/SessionId to achieve this.

In [ ]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

The status *204 No Content* means the session object has been deleted.

* Finally, reset your applications YAML files and delete the kubeconfig file

In [ ]:
#reset the application deployment name in the YAML file
sed -i "s/${username}/example/g" $helloWorldApp
sed -i "s/${username}/example/g" $tensorFlowApp
cat $helloWorldApp
cat $tensorFlowApp

## Summary

In this tutorial, we have shown you how **tenant users** can deploy, programmatically, cloud native stateless, microservices applications, and non-cloud native distributed stateful AI/analytics KubeDirector applications on Kubernetes clusters managed by the HPE Ezmeral Runtime Enterprise. You also used the standard Kubernetes command line (kubectl) to interact with the Kubernetes cluster in the context of your tenant user account. 

* [Conclusion](3-WKSHP-Conclusion.ipynb)